# Parameter Estimation with MCMC

This notebook demonstrates how to perform cosmological parameter estimation using MCMC techniques.

## Setup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from cmb_analysis.cosmology import LCDM
from cmb_analysis.analysis import PowerSpectrumCalculator, MCMCAnalysis
from cmb_analysis.visualization import CMBPlotter, MCMCDiagnostics

%matplotlib inline
plt.style.use('seaborn-v0_8-paper')

## 1. Generate Mock Data

First, let's generate some mock CMB data with realistic noise.

In [ ]:
# True parameters
true_params = {
    'H0': 67.32,
    'omega_b': 0.02237,
    'omega_cdm': 0.1200,
    'tau': 0.0544,
    'ns': 0.9649,
    'ln10As': 3.044
}

# Generate mock data
calculator = PowerSpectrumCalculator()
cl_tt, cl_ee, cl_te = calculator.compute_all_spectra(true_params)

# Add noise
noise_level = 0.01
mock_data = {
    'tt_data': cl_tt * (1 + noise_level * np.random.randn(len(cl_tt))),
    'te_data': cl_te * (1 + noise_level * np.random.randn(len(cl_te))),
    'ee_data': cl_ee * (1 + noise_level * np.random.randn(len(cl_ee))),
    'tt_error': cl_tt * noise_level,
    'te_error': cl_te * noise_level,
    'ee_error': cl_ee * noise_level
}

## 2. Set Up MCMC Analysis

Now we'll configure and run the MCMC analysis.

In [ ]:
# Initialize MCMC analysis
mcmc = MCMCAnalysis(calculator, mock_data, true_params)

# Run MCMC
print("Running MCMC...")
mcmc.run_mcmc(progress=True)

## 3. Analyze MCMC Results

Let's examine the MCMC chain and parameter constraints.

In [ ]:
# Get chain
chain = mcmc.get_chain()

# Initialize diagnostics
diagnostics = MCMCDiagnostics()

# Plot chain evolution
fig = diagnostics.plot_chain_evolution(
    chain,
    list(true_params.keys())
)
plt.show()

# Plot corner plot
fig = diagnostics.plot_corner(
    chain[1000:].reshape(-1, len(true_params)),
    list(true_params.keys()),
    truths=list(true_params.values())
)
plt.show()

## 4. Compute Parameter Constraints

Let's calculate the parameter constraints from our MCMC chains.

In [ ]:
# Get statistics
stats = mcmc.compute_statistics()

print("Parameter Constraints:")
for param, values in stats.items():
    print(f"{param}: {values['mean']:.4f} ± {values['std']:.4f}")

# Compare with true values
print("\nComparison with true values:")
for param in true_params:
    diff = abs(stats[param]['mean'] - true_params[param]) / stats[param]['std']
    print(f"{param}: {diff:.2f}σ from true value")

## 5. Assess Convergence

Let's check the convergence of our MCMC chains.

In [ ]:
# Compute convergence diagnostics
conv_stats = mcmc.compute_convergence_diagnostics()

print("Convergence Diagnostics:")
print(f"Mean acceptance fraction: {conv_stats['acceptance_fraction']:.3f}\n")

print("Gelman-Rubin Statistics:")
for param, gr in conv_stats['gelman_rubin'].items():
    print(f"{param}: {gr:.3f}")

print("\nEffective Sample Size:")
for param, n_eff in conv_stats['effective_samples'].items():
    print(f"{param}: {n_eff:.0f}")

## 6. Plot Best-Fit Results

Finally, let's compare our best-fit model with the data.

In [ ]:
# Get best-fit parameters
best_fit_params = mcmc.get_best_fit()

# Compute best-fit spectra
cl_tt_best, cl_ee_best, cl_te_best = calculator.compute_all_spectra(best_fit_params)

# Plot comparison
plotter = CMBPlotter()
theory = {
    'cl_tt': cl_tt_best,
    'cl_ee': cl_ee_best,
    'cl_te': cl_te_best
}
data = {
    'cl_tt': mock_data['tt_data'],
    'cl_ee': mock_data['ee_data'],
    'cl_te': mock_data['te_data']
}
errors = {
    'cl_tt': mock_data['tt_error'],
    'cl_ee': mock_data['ee_error'],
    'cl_te': mock_data['te_error']
}

fig = plotter.plot_power_spectra(theory, data, errors)
plt.show()

# Plot residuals
fig = plotter.plot_residuals(theory, data, errors)
plt.show()

## 7. Conclusion

In this notebook, we've demonstrated:

1. Setting up and running MCMC analysis
2. Examining chain convergence
3. Computing parameter constraints
4. Visualizing results

The results show good agreement with our input parameters, demonstrating the effectiveness of our parameter estimation pipeline.